# Perfil de Densidade a partir da simulação de um poço</br>

### Script desenvolvido por Victor Carreira
### Colaboração de Rodrigos Mota e Bijani

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import pylab as py
import skimage as sk
import imageio as io
from scipy.misc import imread, imsave, imresize
import plotly as ply
import plotly.plotly as plyy
import plotly.graph_objs as go
import pandas as pd
import random

In [ ]:
#Leitura da imagem (.png) com o modelo geológico
#ma = io.imread("../figs/amazonas.png", as_gray=False, pilmode="RGBA") # verificar no inkscape!
ma = io.imread("../figs/amazonas.png", as_gray=False, pilmode="RGB")
plt.style.use(['classic'])
py.rcParams['figure.figsize'] = (15.0, 20.0) #new figure dimension
plt.title("Modelo - Bacia do Amazonas")
plt.imshow(ma)
plt.grid()
plt.show()

In [ ]:
#Criando os vetores de coordenadas (xi), com 7000m e o vetor de profundidade (zi), com 2000m.
hy = np.shape(ma)[0]
hx = np.shape(ma)[1]
hz = np.shape(ma)[2]

xi = np.linspace(0.0, 7000.0, hx, endpoint=True)
zi = np.linspace(0.0, 2000.0, hy, endpoint=True)

In [ ]:
# para fazer plots dentro do proprio script sem a necessidade de conexao com a internet:
ply.offline.init_notebook_mode(connected=True)

# funcao go.Contour faz um countour plot usando como eixo x, o xi criado para coordenadas, como o eixo y, o zi
# criado para profunidades, e como o eixo z o plot de valores RGBA da matriz ma.
# além disso, na função go.countour está listando as configurações pernsonalizadas para o colorbar

imagem = go.Heatmap(z=ma[:,:,0],
                         x=xi,
                         y=zi,
                    colorbar=dict(
                        title='RGBA',
                        titleside='right',
                        titlefont=dict(
                            size=14,
                            family='Arial, sans-serif'
                        ),
                        thickness=25,
                        thicknessmode='pixels',
                        len=0.9,
                        lenmode='fraction',
                        outlinewidth=0
                    )
                   )
          

# a função go.Layout personaliza o layout do gráfico, no caso, colocando título nos eixos.
# "autorange=reversed", da o comando para inverter o eixo y, no caso as profundidades, começando com 0 e os
# aumentando p/ baixo.
layout = go.Layout(
            xaxis= dict(title='Distância',
                        side ='top',
                        color = 'white'
                   ),
            yaxis= dict(title='Profundidade',
                        autorange='reversed',
                        color = 'white'
                   ),
            # define o tamanho do gráfico, altura e largura, e o tamanhado da margem (l=left, r=right, b=bottom, t=top)
            autosize=False,
            width=1000,
            height=500,
            margin=go.layout.Margin(
                l=50,
                r=30,
                b=20,
                t=60
            ),
            paper_bgcolor='#7f7f7f', #cor do background da margem
            plot_bgcolor='#c7c7c7' #cor do backgroud do gráfico
    )

# a função go.FigureWidget cria o a figura com os parametros utilizados na go.Countour e o layout na go.Layout
fig = go.FigureWidget(data=[imagem], layout=layout)

#Selecionando as configurações dos dados de fig
teste = fig.data[0]
x_data = []
y_data = []

# criando a função de callback
# para selecionar o ponto inicial e final da perfuração simulada do poço sintético 
def select_point(trace, points, selector):
    x = list(teste.x)
    y = list(teste.y)
    for i in points.point_inds:
        y_data.append(zi[i[0]])
        x_data.append(xi[i[1]])
        print(points.point_inds)

# Sempre que clicar no gráfico chamará a função de callback
teste.on_click(select_point)

# plot a figura na tela:
fig

####### É NECESSÁRIO ESCOLHER DOIS PONTOS CLICANDO NO GRÁFICO GERADO PARA RODAR O RESTANTE DO SCRIPT #######

In [ ]:
# para fazer plots dentro do proprio script sem a necessidade de conexao com a internet:
ply.offline.init_notebook_mode(connected=True)

# funcao go.Contour faz um countour plot usando como eixo x, o xi criado para coordenadas, como o eixo y, o zi
# criado para profunidades, e como o eixo z o plot de valores RGBA da matriz ma.
# além disso, na função go.countour está listando as configurações pernsonalizadas para o colorbar

imagem = go.Heatmap(z=ma[:,:,0],
                         x=xi,
                         y=zi,
                    colorbar=dict(
                        title='RGBA',
                        titleside='right',
                        titlefont=dict(
                            size=14,
                            family='Arial, sans-serif'
                        ),
                        thickness=25,
                        thicknessmode='pixels',
                        len=0.9,
                        lenmode='fraction',
                        outlinewidth=0
                    )
                   )
          

# a função go.Layout personaliza o layout do gráfico, no caso, colocando título nos eixos.
# "autorange=reversed", da o comando para inverter o eixo y, no caso as profundidades, começando com 0 e os
# aumentando p/ baixo.
layout = go.Layout(
            xaxis= dict(title='Distância',
                        side ='top',
                        color = 'white'
                   ),
            yaxis= dict(title='Profundidade',
                        autorange='reversed',
                        color = 'white'
                   ),
            # define o tamanho do gráfico, altura e largura, e o tamanhado da margem (l=left, r=right, b=bottom, t=top)
            autosize=False,
            width=1000,
            height=500,
            margin=go.layout.Margin(
                l=50,
                r=30,
                b=20,
                t=60
            ),
            paper_bgcolor='#7f7f7f', #cor do background da margem
            plot_bgcolor='#c7c7c7' #cor do backgroud do gráfico
    )

imagem2 = go.Scatter(x=x_data,y=y_data)

data = [imagem, imagem2]

fig = go.FigureWidget(data=data, layout=layout)

# plot a figura na tela:
fig

In [ ]:
#apenas rodar quando escolher dois pontos clicando no gráfico acima
#definindo a dimensão do poço sintético
mb = (ma[y_data[0]-1:y_data[1], x_data[0]-1:x_data[0]])

## Estabelecendo critério de soma das rbga para construção do poço sintético:

In [ ]:
# calculando as somas das cores para definir
lito = [] # soma das cores do rbga = r+b+g+a:
for i in range( len(mb) ):
    lito.append(sum(mb[i][0]))

### Leitura do banco de dados de input, fornecido pelo usuario (litologias, RBG, rho)

In [ ]:
cab = ['lito', 'R','G','B','A', 'rho']
data = pd.read_csv('../dados/input_usuario.txt', sep='\s+', names=cab, header=0 )
print(data)

In [ ]:
# Somando os RBGA:
soma = data.R + data.G + data.B + data.A
#adicionando a coluna 'soma' na tabela de dados:
data['soma']= soma

In [ ]:
# construindo o perfil de densidade:
rhob = [0.0]*len(lito)

for i in range(len(lito)):
    if lito[i] == data.soma[0]: # folhelho
        rhob[i] = data.rho[0]
        
    if lito[i] == data.soma[1]: # A1
        rhob[i] = data.rho[1]
        
    if lito[i] == data.soma[2]: # A2
        rhob[i] = data.rho[2]
        
    if lito[i] == data.soma[3]: # Marga
        rhob[i] = data.rho[3]
        
    if lito[i] == data.soma[4]: # Basalto
        rhob[i] = data.rho[4]
print(rhob)

In [ ]:
## Adiconando ruído randômico gaussiano normal:
rhob_noise = np.zeros( len(rhob) )

for i in range(len(rhob)):
    rhob_noise[i] = random.gauss(rhob[i], 0.6)

In [ ]:
#Criando o vetor de profundidade:
prof = np.linspace(zi[y_data[0]], zi[y_data[1]], len(rhob))

In [ ]:
# para fazer plots dentro do proprio script sem a necessidade de conexao com a internet:
ply.offline.init_notebook_mode(connected=True)

# funcao go.Scatter faz um scatter plot usando como eixo x, o singnal_RHObm criado com os dados de rhob e
# adicionado ruído, como o eixo y, a prof criada para profunidades

rhob_graph = [go.Scatter(x=rhob_noise, y=prof)]

# Define o layout do plot, como título dos eixos, posição dos eixos, cor da letra, etc
layout = go.Layout(
    xaxis= dict(title='Propriedade Fisica',
                side ='top',
                color = 'white'
               ),
    yaxis= dict(title='Profundidade',
                autorange='reversed',
                color = 'white'
               ),
    # define o tamanho do gráfico, altura e largura, e o tamanhado da margem (l=left, r=right, b=bottom, t=top)
    autosize=False,
    width=400,
    height=800,
    margin=go.layout.Margin(
        l=50,
        r=30,
        b=20,
        t=60
        ),
    paper_bgcolor='#7f7f7f', #cor do background da margem
    plot_bgcolor='#c7c7c7' #cor do backgroud do gráfico
    )

fig = go.FigureWidget(data=rhob_graph, layout=layout)

fig

## TODO:
### 1) Usar o plotly para plotar o poço interativamente, ou seja, no "ato de perfilar" com os cliques do mouse;   
### 2) Criar poços direcionais (futuro);
### 3) Como seria fazer vários poços na mesma janela de plot?
### 4) Limpar o codigo criando funções (plolty, da soma dos RBG)
### 5) fazer o perfil de VS a partir de rhob (Gardner)

# Para saber mais sobre a go.Countour:

### https://plot.ly/python/contour-plots/
### https://plot.ly/python/click-events/#update-points-using-a-click-callback